# Processador


In [1]:
import xml.etree.ElementTree as ET

tree = ET.parse('../data/cfquery.xml')
root = tree.getroot()


In [2]:
from unidecode import unidecode
import pandas as pd

def clean_string(s):
    return unidecode(s.upper())

df_consultas = pd.DataFrame(columns=['query_number', 'query_text'])

dict_list = []

for query in root:

    query_number = query.find('QueryNumber').text
    query_text = query.find('QueryText').text

    # Clean query text
    query_text = clean_string(query_text)

    dict_list.append({'query_number': query_number, 'query_text': query_text})

df_consultas = pd.DataFrame(dict_list)

df_consultas.to_csv('../results/consultas.csv', index=False)

In [3]:
df_consultas

,query_number,query_text
0,00001,WHAT ARE THE EFFECTS OF CALCIUM ON THE PHYSICA...
1,00002,CAN ONE DISTINGUISH BETWEEN THE EFFECTS OF MUC...
2,00003,HOW ARE SALIVARY GLYCOPROTEINS FROM CF PATIENT...
3,00004,WHAT IS THE LIPID COMPOSITION OF CF RESPIRATOR...
4,00005,IS CF MUCUS ABNORMAL?\n
...,...,...
94,00096,IS THERE AN INCREASED INCIDENCE OF DENTAL PROB...
95,00097,IS OXYGEN TRANSPORT BY RED BLOOD CELLS ABNORMA...
96,00098,WHAT ARE THE EFFECTS OF CF ON THE DEVELOPMENT ...
97,00099,ARE THERE ABNORMALITIES OF TASTE IN CF PATIENT...


In [6]:
import pandas as pd
import xml.etree.ElementTree as ET
from utils import read_config, clean_string



# Read the XML file
tree = ET.parse('../data/cfquery.xml')
root = tree.getroot()


In [9]:
df_esperados = pd.DataFrame(columns=['query_number', 'query_text'])

dict_list = []

def get_votes(string):
    
    votes = string.string.strip()
    count = 0
    for vote in votes:
        count += int(vote)
    return count


for query in root:

    # Get query number and query text
    query_number = query.find('QueryNumber').text
    
    records = query.find('Records')

    for doc in records:
        print(doc.text)
        print(doc.attrib.get('score'))

139
1222
151
2211
166
0001
311
0001
370
1010
392
0001
439
0001
440
0011
441
2122
454
0100
461
1121
502
0002
503
1000
505
0001
520
1010
522
0001
526
1011
527
1001
533
2222
593
1000
619
0001
737
0100
742
0100
789
0001
827
0010
835
0001
861
0001
875
1121
891
0001
921
1010
922
1010
1175
0100
1185
0001
1222
0001
169
1000
434
1001
454
0100
498
1000
499
1000
592
0002
875
1011
23
1000
40
0010
139
2122
190
0001
221
0001
246
0001
309
1000
311
0011
325
0001
345
0010
347
0010
356
0010
370
1001
374
0001
375
2221
439
0001
440
1120
454
1220
515
1000
520
0010
524
0010
526
1010
527
1220
533
0001
535
0010
560
0010
561
0001
571
0010
584
0010
604
0002
623
0010
633
2222
733
1000
742
1010
854
0100
856
2112
950
0010
967
0001
1144
1111
1161
0001
1172
1000
1175
0001
1196
0002
503
0001
538
0100
539
0100
540
0100
553
0001
604
2222
669
1010
711
2122
876
2222
23
2220
47
2221
50
0001
60
0001
114
0011
132
0001
135
0001
139
0010
151
2222
169
0001
189
0001
190
0001
197
0001
200
0001
226
0001
256
2211
265
0010
297
0001

# Gerador Lista Invertida

In [32]:
import pandas as pd
import xml.etree.ElementTree as ET

def read_cfg_file(file:str) -> dict:
    data = {}
    
    with open(file, 'r') as cfg_file:
        for line in cfg_file:
            line = line.strip()
            if '=' in line:
                key, value = line.split('=')

                if key in data.keys():
                    data[key].append(value)
                else:
                    data[key] = [value]

    return data

cfg = read_cfg_file('../config/GLI.CFG')
read_files = cfg['LEIA']

In [68]:
from unidecode import unidecode

def coalesce(*args):
    for arg in args:
        if arg is not None:
            return unidecode(arg.text).upper()
    return ''  

In [70]:
generator_list = {}

for file in read_files:
    tree = ET.parse('../data/' + file)
    root = tree.getroot()

    for record in root:
        record_num = record.find('RECORDNUM').text
        record_text = coalesce(record.find('ABSTRACT'), record.find('EXTRACT'))

        for word in record_text.split():
            if word in generator_list.keys():
                generator_list[word].append(record_num)
            else:
                generator_list[word] = [record_num]
generator_list_df = pd.DataFrame({'WORDS': generator_list.keys(), 'RECORDS': generator_list.values()})
generator_list_df.to_csv('../results/gerador.csv', index=False)

# Indexador


In [82]:
config = read_cfg_file('../config/INDEX.CFG')

read_files = config['LEIA']

list_of_words = pd.read_csv('../results/' + read_files[0], sep=';')


In [114]:
from collections import Counter
import numpy as np

max_freq_term = {}
nis = {}
freq_words = {}
weights = {}


for index, row in list_of_words.iterrows():
    word = row['WORDS']
    docs = eval(row['RECORDS'])
    doc_appearences= Counter(docs)
    nis[word] = len(doc_appearences.keys())

    freq_words[word] = doc_appearences
    for doc in doc_appearences.keys():
        if doc in max_freq_term.keys():
            max_freq_term[doc] = max(max_freq_term[doc], doc_appearences[doc])
        else:
            max_freq_term[doc] = doc_appearences[doc]
       

N = len(max_freq_term.keys())
for word in nis.keys():
    for doc in max_freq_term.keys():
        # Normalized frequency of word in doc
        fij = freq_words[word].get(doc,0)/max_freq_term[doc]
        # Inverse document frequency
        idf = np.log(N/nis[word])

        weight = fij * idf

        if word in weights.keys():
            weights[word].append(weight)
        else:
            weights[word] = [weight]

TypeError: unsupported operand type(s) for /: 'NoneType' and 'int'

In [112]:
freq_words['THE'].get('00001 ')

20